In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import random
import struct
import torch
import torchvision
from torchvision import transforms,datasets
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import PIL
from PIL import Image
import shutil
from torch.utils.data.sampler import SubsetRandomSampler
import torch.backends.cudnn as cudnn
import torchvision.models as models
from tqdm import tqdm

In [3]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [5]:
model = torchvision.models.resnet18(pretrained=True, progress=True)

In [11]:
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)

In [78]:
model.train()
model.to(device)
model.double()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [59]:
#creating dataset 
#we will reshape all the training images to same shape of (224,224,3)
DATA_PATH = r'C:/Users/occisor/Downloads/Synergy_tasks/dataset'
for folder in os.listdir(DATA_PATH):
    path_to_class = os.path.join(DATA_PATH,folder).replace("\\","/")
    try:
        for img_file in os.listdir(path_to_class):
            path_to_image = os.path.join(path_to_class,img_file).replace("\\","/")
            #l = len(path_to_image)
            #if path_to_image[l-3:l] != 'jpg':
             #   continue
            img = Image.open(path_to_image)
            img = img.resize((224,224), Image.ANTIALIAS)
            save_path = r'C:/Users/occisor/Downloads/Synergy_tasks/dataset_new'
            save_path = os.path.join(save_path,folder).replace("\\","/")
            save_to_class = os.path.join(save_path,img_file).replace("\\","/")
            img.save(save_to_class) 
    except OSError:
        pass

D:\anaconda\envs\tensorflow\lib\site-packages\PIL\Image.py:2817: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


In [62]:
path_dataset_new = r'C:\Users\occisor\Downloads\Synergy_tasks\dataset_new'

In [64]:
image_datasets =datasets.ImageFolder(path_dataset_new,transform_train)

batch_size = 10
#validation_split = .2
shuffle_dataset = True
#random_seed= 20

    # Creating data indices for training and validation splits:
# dataset_size = len(image_datasets)
# indices = list(range(dataset_size))
# split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
#     train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
#train_sampler = SubsetRandomSampler(train_indices)
#valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, 
                                               shuffle=shuffle_dataset)
# validation_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size,
#                                                     sampler=valid_sampler)

In [65]:
torch.backends.cudnn.deterministic = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [79]:
input = Variable(torch.Tensor(1,3,224,224))
input = input.to(device)
input = input.double()
output = model.forward(input)

In [80]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [88]:
for epoch in tqdm(range(10)):
    for data in train_loader:
        print(1)
        inputs,labels=data
        inputs = inputs.to(device)
        inputs=inputs.double()
        inputs=Variable(inputs)
        #labels = one_hot_embedding(labels,5)
        labels = labels.to(device)
        labels = labels.double()
        labels=Variable(labels)
        outputs=model.forward(inputs)
        labels = labels.type(torch.cuda.LongTensor)
        loss=criterion(outputs,labels)
        model.zero_grad()
        loss.backward()
        optimizer.step()


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1



 10%|████████▎                                                                          | 1/10 [01:29<13:23, 89.24s/it]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1



 20%|████████████████▌                                                                  | 2/10 [03:12<12:27, 93.47s/it]

...


 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [17:21<04:38, 139.06s/it]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1



 90%|█████████████████████████████████████████████████████████████████████████▊        | 9/10 [19:49<02:21, 141.57s/it]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1



100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [22:12<00:00, 133.24s/it]


In [132]:
test_path = r'C:\Users\occisor\Downloads\Synergy_tasks\test_data'
for folder in os.listdir(test_path):
    path_to_class = os.path.join(test_path,folder)
    try:
        for img_file in os.listdir(path_to_class):
            path_to_image = os.path.join(path_to_class,img_file)
            #l = len(path_to_image)
            #if path_to_image[l-3:l] != 'jpg':
             #   continue
            img = cv2.imread(path_to_image)
            img  = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = img.resize((224,224), Image.ANTIALIAS)
            save_path = r'C:\Users\occisor\Downloads\Synergy_tasks\test_data_new'
            save_path = os.path.join(save_path,folder)
            save_to_class = os.path.join(save_path,img_file)
            img.save(save_to_class) 
    except OSError:
        pass

In [133]:
test_data_path = r'C:\Users\occisor\Downloads\Synergy_tasks\test_data_new'
test_data = datasets.ImageFolder(test_data_path,transform_train)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=1,
                                                     shuffle = True)

In [134]:
acc = 0
Total_images = 0
for data in validation_loader:
    inputs,labels=data
    Total_images += len(inputs)
    inputs = inputs.to(device)
    inputs=inputs.double()
    inputs=Variable(inputs)
    labels = labels.to(device)
    labels = labels.double()
    labels=Variable(labels)
    outputs=model.forward(inputs)
    if outputs.argmax() == labels.argmax():
        acc+=1

In [135]:
accuracy = acc/Total_images

In [137]:
accuracy

1.0